# Mini-projet groupe 2 : interpolation de données manquantes
Contexte : dans le cadre d'acquisition de données sismiques, des capteurs (récepteurs) sont positionnés le long d'une ligne à la surface de la terre. Chacun enregistre au cours du temps le deplacement des particules (signal oscillant) qui varie lors du passage d'une onde acoustique ou élastique. Cet enregistrement s'appelle une **trace**.

En pratique, il arrive que certains capteurs sont défaillants ou encore qu'ils ne sont pas présents dans certaines zones (pour des problèmes d'accessibilité par exemple). L'objectif principal du mini-projet est de reconstituer les traces manquantes.

Hypotheses : nous ferons l'hypothèse importante que les **signaux correspondent à la superposition lineaire d'ondes planes** dans les enregistrements (voir l'exemple fourni plus bas avec 3 événements).

Notions abordées : 
* Analyse des signaux après transformée de Fourier 2d
* Analyse de signaux aliasés
* Régularisation et formulation sous forme de problème inverse

Pour cela, deux approches sont considerées :
* Approche 1 : transformée de Fourier 2d, identification des événements principaux, définition d'un masque et reconstitution des traces manquantes
* Approche 2 : suite de l'approche 1 pour s'assurer que la prédiction respecte bien les observables. Elle se fait au travers de la résolution d'un problème inverse avec ajout de régularisation

Attentes :
* Développement des approches 1 et 2 sur des données fournies ici
* Discusssion des limites des approches sur des exemples que vous pourrez créer vous-mêmes

Consignes :
* **Très important** : pour toutes les figures, bien indiquer les axes avec les bonnes unités et des labels de taille lisibles

In [1]:
import numpy as np
import matplotlib.pyplot as plt
# Pour la question 4
from scipy.optimize import minimize
# %matplotlib auto
%matplotlib inline

Using matplotlib backend: TkAgg


In [2]:
# Labelsize (display)
labelsize   = 14
    
# Load the input data
# Input data (with missing traces)
gpanel = np.load('gpanel.npy')
# Dense data (with all traces, only for comparison)
panel_dense = np.load('panel_dense.npy')
nt, nx = gpanel.shape

# Vertical axis -- time
dt   = 3.125e-3  # increment (s)
at   = np.linspace(0,dt*(nt-1),nt)
# Horizontal axis -- space
dx   = 10. # increment (m)
ax   = np.linspace(0,dx*(nx-1),nx)

In [ ]:
# Display of the input data
vmax    = np.max(np.abs(gpanel))
fig = plt.figure()
av = plt.subplot(111)
plt.imshow(gpanel,extent=[ax[0],ax[-1],at[-1],at[0]],aspect='auto')
plt.title('Observables (avec traces manquantes mises à zero)', fontsize = labelsize)
av.set_ylabel("Time (s)", fontsize = labelsize)
av.set_xlabel("Position x (m)", fontsize = labelsize)
av.tick_params(axis='both', which='major', labelsize=labelsize)
plt.clim([-vmax,vmax])
plt.set_cmap('bwr')

fig = plt.figure()
av = plt.subplot(111)
plt.imshow(panel_dense,extent=[ax[0],ax[-1],at[-1],at[0]],aspect='auto')
plt.title('Observables (seulement pour comparaison)', fontsize = labelsize)
av.set_ylabel("Time (s)", fontsize = labelsize)
av.set_xlabel("Position x (m)", fontsize = labelsize)
av.tick_params(axis='both', which='major', labelsize=labelsize)
plt.clim([-vmax,vmax])
plt.set_cmap('bwr')

# Question 1 : analyse des données dans le domaine de Fourier 2d
Représenter la transformée de Fourier 2d et bien préciser les axes. Indiquer à quoi correspond la fréquence maximale et le nombre d'onde maximal. Faire le lien entre les événements dans l'espace $(t,x)$ et ceux dans l'espace de Fourier. Expliquer ce qu'il se passe aux bords dans le domaine de Fourier. Ne pas hésiter à zoomer dans le domaine de Fourier.

In [4]:
# plt.figure()
data_2D_fft = np.fft.fft2(gpanel)
data_fft2D_orig = np.copy(data_2D_fft)
print(data_2D_fft.shape)

freq_axis_0 = np.fft.fftfreq(data_2D_fft.shape[0], dt)
freq_axis_1 = np.fft.fftfreq(data_2D_fft.shape[1], dt)
print("0 :", freq_axis_0.size)
print("1 :", freq_axis_1.size)
print()
print("fréquence maximale : ", np.max(freq_axis_0))
print("nb d'onde maximal : ", np.max(freq_axis_1))
print(1/dt)

# plt.imshow(np.abs(data_2D_fft),extent=[np.abs(freq_axis_0)[0],np.abs(freq_axis_0)[-1],np.abs(freq_axis_1)[-1],np.abs(freq_axis_1)[0]], aspect = 'auto')

(400, 30)
0 : 400
1 : 30

fréquence maximale :  159.20000000000002
nb d'onde maximal :  149.33333333333331
320.0


In [5]:
plt.figure()
plt.imshow(np.real(data_2D_fft),extent=[np.abs(freq_axis_0)[0],np.abs(freq_axis_0)[-1],np.abs(freq_axis_1)[-1],np.abs(freq_axis_1)[0]], aspect = 'auto')



In [7]:
plt.figure()
plt.contour(freq_axis_1,freq_axis_0,np.imag(data_2D_fft))

In [8]:
freq_axis_0 = np.fft.fftfreq(data_2D_fft.shape[0], dt)
print(freq_axis_0.size)
freq_axis_0[:freq_axis_0.size//2];

400


In [11]:
plt.figure()
plt.plot(np.abs(freq_axis_0), data_fft2D_orig[:,0]);
plt.plot(np.abs(freq_axis_0), data_2D_fft[:,0]);
# temporel

In [ ]:
freq_axis_1 = np.fft.fftfreq(data_fft2D_orig.shape[1], dt)
freq_axis_1.size

In [ ]:
plt.plot(np.abs(freq_axis_1), data_fft2D_orig[0,:]);
plt.plot(np.abs(freq_axis_1), data_2D_fft[0,:]);
# spatial

In [12]:
data_fft2D_orig

array([[ -5.28580267 +0.j        ,  -3.8453576  +1.70579876j,
          7.97028451 -6.18236314j, ...,  -7.07834968+12.96265657j,
          7.97028451 +6.18236314j,  -3.8453576  -1.70579876j],
       [-11.97014868-13.25019701j,   0.33836002 +1.85379365j,
          5.94114556 -1.2175584j , ...,  -5.53771073 +1.96482001j,
          5.69605387-11.51253642j,   4.90220932 -6.1528001j ],
       [ 12.49315162-10.44586898j, -12.72275388+10.84051218j,
         -5.38328269 -1.94623059j, ..., -20.55911436-11.78849115j,
         -6.68044309 +0.57405224j, -41.56350289+28.66587763j],
       ...,
       [-15.02143699+16.91758725j,  12.08152742+62.19723496j,
        -28.96645542-60.01549156j, ...,  -7.69683846+20.02217919j,
         19.42130343+12.98058166j,  -1.04387776+23.77246595j],
       [ 12.49315162+10.44586898j, -41.56350289-28.66587763j,
         -6.68044309 -0.57405224j, ...,   7.5900022 -28.09502051j,
         -5.38328269 +1.94623059j, -12.72275388-10.84051218j],
       [-11.97014868+13.2501

In [20]:
# data_2D_fft[15:, :] = 0
data_2D_fft = np.copy(data_fft2D_orig)
data_2D_fft[:, 10:] = 0
data_2D_fft

array([[ -5.28580267 +0.j        ,  -3.8453576  +1.70579876j,
          7.97028451 -6.18236314j, ...,   0.         +0.j        ,
          0.         +0.j        ,   0.         +0.j        ],
       [-11.97014868-13.25019701j,   0.33836002 +1.85379365j,
          5.94114556 -1.2175584j , ...,   0.         +0.j        ,
          0.         +0.j        ,   0.         +0.j        ],
       [ 12.49315162-10.44586898j, -12.72275388+10.84051218j,
         -5.38328269 -1.94623059j, ...,   0.         +0.j        ,
          0.         +0.j        ,   0.         +0.j        ],
       ...,
       [-15.02143699+16.91758725j,  12.08152742+62.19723496j,
        -28.96645542-60.01549156j, ...,   0.         +0.j        ,
          0.         +0.j        ,   0.         +0.j        ],
       [ 12.49315162+10.44586898j, -41.56350289-28.66587763j,
         -6.68044309 -0.57405224j, ...,   0.         +0.j        ,
          0.         +0.j        ,   0.         +0.j        ],
       [-11.97014868+13.2501

In [21]:
plt.figure()
plt.subplot(121).title.set_text("original")
plt.imshow(np.real(data_fft2D_orig),extent=[np.abs(freq_axis_0)[0],np.abs(freq_axis_0)[-1],np.abs(freq_axis_1)[-1],np.abs(freq_axis_1)[0]], aspect = 'auto')

plt.subplot(122).title.set_text("reconstitué")
plt.imshow(np.real(data_2D_fft),extent=[np.abs(freq_axis_0)[0],np.abs(freq_axis_0)[-1],np.abs(freq_axis_1)[-1],np.abs(freq_axis_1)[0]], aspect = 'auto');

In [22]:
inv_fft2 = np.fft.ifft2(data_2D_fft)

plt.figure()
plt.subplot(121).title.set_text("original")
plt.imshow(gpanel,extent=[ax[0],ax[-1],at[-1],at[0]],aspect='auto')
plt.subplot(122).title.set_text("reconstitué")
plt.imshow(np.real(inv_fft2), aspect = 'auto');



# Question 2 : définition d'un masque dans le domaine de Fourier 2d
On note $f$ la fréquence et $k_x$ le nombre d'onde. Dans le domaine de Fourier 2d, on souhaite definir un masque avec des 1 qui couvrent les événements principaux. On va supposer que les événements dans le domaine $(t,x)$ sont linéaires. Montrer qu'il en est de même dans le domaine $(f,k_x)$. A partir de $(f,k_x)=(0,0)$, faire des sommations selon différentes pentes (dans le domaine de Fourier, apres avoir pris le module des valeurs complexes) et tracer la valeur de ces sommes en fonction de la pente. On s'attend à ce que 3 valeurs ressortent. Calculer une fonction qui sélectionne automatiquement ces valeurs et construire un masque dans le domaine de Fourier 2d qui vaut 1 autour de ces trois valeurs principales et 0 en dehors.

In [ ]:
plt.figure()
plt.imshow(np.abs(data_fft2D_orig), aspect="auto")#,extent=[np.abs(freq_axis_0)[0],np.abs(freq_axis_0)[-1],np.abs(freq_axis_1)[-1],np.abs(freq_axis_1)[0]], aspect = 'auto');
# print(np.shape(np.abs(data_fft2D_orig)))
# print(np.abs(data_fft2D_orig))
# print(np.abs(data_fft2D_orig)[398, 0])
# print(np.abs(freq_axis_0))

# Question 3 : application du masque et analyse des résultats
Appliquer le masque (sur les données après transformée de Fourier 2d, sans prendre le module des valeurs), puis appliquer la transformée de Fourier inverse. Commenter les résultats (comparaison à la fois dans les domaines $(t,x)$ et $(f,k_x)$). Si nécessaire, revenir à la question 2 pour changer le masque et avoir une meilleure interpolation des données. Expliquer ces changements.

# Question 4 : amélioration de l'approche 1
Le résultat de l'approche 1 montre en pratique que les données prédites après transformée de Fourier inverse 2d ne calent pas parfaitement aux données observées (pour les traces non nulles). C'est un inconvénient, mais d'un autre côté, ceci offre la possibilité d'aller plus loin forcer un bon calage.

Pour cela, nous définissons une fonction objective

$$
J[p] = \frac{1}{2}||T p - p^\mathrm{obs}||_2^2 + \frac{\alpha}{2} || F p ||_M^2
$$

où $p(t,x)$ sont les signaux (sur toutes les traces), $T$ l'operateur qui vaut 1 pour des traces existantes et 0 sinon, $p(t,x)^\mathrm{obs}$ les observables (i.e. pour les traces non nulles), $F$ la transformee de Fourier 2d. La norme $||p||_2^2 = \sum_x \sum_t |p(t,x)|^2$ et $||q||_M^2= \sum_x \sum_t M(f,k_x)|q(f,k_x)|^2$, avec $M$ un masque. Le poids $\alpha$ est une pondération entre les deux termes.

On cherche à minimiser $J$. Le meilleur $p$ sera celui qui minimise l'écart aux données (premier terme) tout en minimisant l'énergie dans le domaine de Fourier 2d. Par rapport au masque $M_0$ des questions 2 et 3, on prend ici $M=1 - M_0$. Expliquer pourquoi.

La minimisation se fait de manière iterative, avec la fonction ```minimize``` de scipy (```from scipy.optimize import minimize```). Parmi plusieurs possibilités, je suggère de prendre une approche de gradient conjugué (CG). La minisation va se faire selon :

```
res = minimize(defj, p0, method='CG', jac=defg, \
        options={'disp': True, 'maxiter':niter})
```

Il faut donc créer une valeur initiale pour $p$ (ici ```p0```), spécifier le nombre d'itérations ```niter``` et surtour définir la fonction objective ```defj()``` et son gradient par rapport à $p$ ```defg()```.

Le gradient est donné par 

$$
\frac{\partial J}{\partial p} = T^t(T p - p^\mathrm{obs}) + \alpha F^t M F p
$$

où $^t$ est l'opérateur transposée. L'application de $T^t$ correspond donc à insérer des traces nulles là où il n'y a pas d'observables et $F^t$ est la transformée de Fourier inverse.

Construire les fonctions ```defj()``` et ```defg()```. Attention, ```minimize``` attend que ```p``` soit sous forme de vecteur. On peut donc utiliser ```np.transpose(p,nt*nx)```.

$\alpha$ est une pondération dont la valeur est à tester (optimalement les deux termes ont a peu près la même valeur).

La valeur initiale ```p0``` de $p$ peut être prise égale à ```gpanel``` (le tableau lu au début du code).

Faire plusieurs itérations. Analyser les résultats à la fois dans le domaine temporel et dans le domaine de Fourier 2d. 

In [ ]:
# Exemple de structure du code
def defj(p):
    """Definition of the objective function"""
    # Retrieve the size
    pan = np.reshape(p,[nt,nx])
    j0  = 0.
    # ADD
    #
    return j0
    
    
def defg(p):
    """Definition of the gradient of the objective function"""
    # Retrieve the size
    pan = np.reshape(p,[nt,nx])
    grd = np.zeros([nt,nx])
    #
    # ADD
    #
    # Return with vector size
    return np.reshape(grd,nt*nx)

In [ ]:
# Iterative mininisation (exemple)
niter = 10 # number of iterations
p0 = np.reshape(np.copy(gpanel),nt*nx)
res = minimize(defj, p0, method='cg', \
                jac=defg, \
                options={'disp': True, 'maxiter':niter})
# Final panel
panelf = np.reshape(res.x,[nt,nx])

# Question 5 : analyse des limites de l'approche
Proposer une analyse des limites des approches 1 et 2, par exemple au travers d'applications sur des données bruitées, qui contiennent des événements qui ne sont pas linéaires, pourquoi pas sur des images dans un contexte très différent. **Cette dernière partie est très ouverte, et j'attends des investigations possiblement très variées.**
